In [477]:
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression,ElasticNet
from sklearn.svm import SVR
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import r2_score,mean_absolute_error
import sklearn
import math
import numpy as np, pandas as pd
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
import matplotlib.pyplot as plt
from sklearn.linear_model import Ridge, RidgeCV, ElasticNet, LassoCV, LassoLarsCV,Lasso
from sklearn.model_selection import cross_val_score
import warnings
warnings.filterwarnings("ignore")

In [478]:
frame=pd.read_excel("COVID-19.xlsx")
frame.drop(["geoId","day","month","year","countryterritoryCode"],axis=1,inplace=True)
frame.columns=[["Date","Confirmed","Deaths","Country","Population"]]
frame.drop("Population",axis=1,inplace=True)
frame.columns = frame.columns.get_level_values(0)
frame["Date"]=pd.to_datetime(frame["Date"],format="%d/%m/%Y")
frame.head()


,Date,Confirmed,Deaths,Country
0,2020-04-10,61,1,Afghanistan
1,2020-04-09,56,3,Afghanistan
2,2020-04-08,30,4,Afghanistan
3,2020-04-07,38,0,Afghanistan
4,2020-04-06,29,2,Afghanistan


In [479]:
frame.dtypes


Date         datetime64[ns]
Confirmed             int64
Deaths                int64
Country              object
dtype: object

In [480]:
frame.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9922 entries, 0 to 9921
Data columns (total 4 columns):
Date         9922 non-null datetime64[ns]
Confirmed    9922 non-null int64
Deaths       9922 non-null int64
Country      9922 non-null object
dtypes: datetime64[ns](1), int64(2), object(1)
memory usage: 310.1+ KB


In [481]:
frame=frame.sort_values(by=["Country","Date"])
frame.head()

,Date,Confirmed,Deaths,Country
91,2019-12-31,0,0,Afghanistan
90,2020-01-01,0,0,Afghanistan
89,2020-01-02,0,0,Afghanistan
88,2020-01-03,0,0,Afghanistan
87,2020-01-04,0,0,Afghanistan


In [482]:
frame["Total_Confirmed"]=frame.groupby("Country")["Confirmed"].cumsum()
frame["Total_Deaths"]=frame.groupby("Country")["Deaths"].cumsum()



In [483]:
df_Bangladesh=frame[frame["Country"]=="Bangladesh"]
df_world=frame[frame["Country"]!="Bangladesh"]

In [484]:
df_Bangladesh.tail()

,Date,Confirmed,Deaths,Country,Total_Confirmed,Total_Deaths
882,2020-04-06,35,1,Bangladesh,123,9
881,2020-04-07,41,3,Bangladesh,164,12
880,2020-04-08,54,5,Bangladesh,218,17
879,2020-04-09,112,3,Bangladesh,330,20
878,2020-04-10,94,1,Bangladesh,424,21


In [485]:
df_world=df_world[df_world["Total_Confirmed"]>0]

In [486]:
df_world['Days'] = (df_world['Date'] - df_world.groupby('Country')['Date'].transform('first'))
df_Bangladesh['Days'] = (df_Bangladesh['Date'] - df_Bangladesh.groupby('Country')['Date'].transform('first'))

In [487]:
df_Bangladesh.head()

,Date,Confirmed,Deaths,Country,Total_Confirmed,Total_Deaths,Days
904,2020-03-09,3,0,Bangladesh,3,0,0 days
903,2020-03-15,0,0,Bangladesh,3,0,6 days
902,2020-03-17,2,0,Bangladesh,5,0,8 days
901,2020-03-18,3,0,Bangladesh,8,0,9 days
900,2020-03-19,2,0,Bangladesh,10,0,10 days


In [488]:
a=df_world["Country"].unique()

In [489]:
df_world["Days"]=df_world["Days"].astype(str)
df_Bangladesh["Days"]=df_Bangladesh["Days"].astype(str)
df_world["Days"] = df_world["Days"].str.split(' ').str[0]
df_Bangladesh["Days"] = df_Bangladesh["Days"].str.split(' ').str[0]
df_world["Days"]=df_world["Days"].astype(int)
df_Bangladesh["Days"]=df_Bangladesh["Days"].astype(int)
df_world["Days"]=df_world["Days"]+1
df_Bangladesh["Days"]=df_Bangladesh["Days"]+1

In [490]:
df_USA=df_world[df_world["Country"]=="United_States_of_America"]
df_USA=df_USA[["Days","Total_Confirmed"]]
df_UK=df_world[df_world["Country"]=="United_Kingdom"]
df_UK=df_UK[["Days","Total_Confirmed"]]
df_Italy=df_world[df_world["Country"]=="Italy"]
df_Italy=df_Italy[["Days","Total_Confirmed"]]
df_Spain=df_world[df_world["Country"]=="Spain"]
df_Spain=df_Spain[["Days","Total_Confirmed"]]
df_France=df_world[df_world["Country"]=="France"]
df_France=df_France[["Days","Total_Confirmed"]]

df_India=df_world[df_world["Country"]=="India"]
df_India=df_India[["Days","Total_Confirmed"]]
df_Pakistan=df_world[df_world["Country"]=="Pakistan"]
df_Pakistan=df_Pakistan[["Days","Total_Confirmed"]]
df_Bangladesh=df_Bangladesh[["Days","Total_Confirmed"]]

In [491]:
df_USA = df_USA.sample(frac=1,random_state=42).reset_index(drop=True)
df_UK = df_UK.sample(frac=1,random_state=42).reset_index(drop=True)
df_Italy = df_Italy.sample(frac=1,random_state=42).reset_index(drop=True)
df_France = df_France.sample(frac=1,random_state=42).reset_index(drop=True)
df_Spain = df_Spain.sample(frac=1,random_state=42).reset_index(drop=True)
df_India = df_India.sample(frac=1,random_state=42).reset_index(drop=True)
df_Pakistan = df_Pakistan.sample(frac=1,random_state=42).reset_index(drop=True)
df_Bangladesh = df_Bangladesh.sample(frac=1,random_state=42).reset_index(drop=True)



In [492]:
df_USA["POP_Density"]=328200000
df_UK["POP_Density"]=66650000
df_Italy["POP_Density"]=260360000
df_France["POP_Density"]=66990000 
df_Spain["POP_Density"]=46940000
df_India["POP_Density"]=1.339e9
df_Bangladesh["POP_Density"]=164700000
df_Pakistan["POP_Density"]=197000000

In [493]:
df=pd.concat([df_USA,df_UK,df_Italy,df_Pakistan,df_India,df_France,df_Spain],ignore_index=True)
df.head()

,Days,Total_Confirmed,POP_Density
0,31,15,328200000.0
1,1,1,328200000.0
2,23,13,328200000.0
3,32,16,328200000.0
4,19,12,328200000.0


In [494]:
df_X=df[["Days","POP_Density"]]
df_Y=df["Total_Confirmed"]

In [495]:
df_Bangladesh_X=df_Bangladesh[["Days","POP_Density"]]
df_Bangladesh_Y=df_Bangladesh["Total_Confirmed"]

In [496]:
from sklearn.kernel_approximation import RBFSampler
rbf_feature = RBFSampler(gamma=1, random_state=1,n_components=10)
df_X = rbf_feature.fit_transform(df_X)
df_Bangladesh_X = rbf_feature.fit_transform(df_Bangladesh_X)
df_Bangladesh_Test= rbf_feature.fit_transform(df_Bangladesh_Test)

In [497]:
print(df_X.shape)
print(df_Bangladesh_X.shape)
print(df_Bangladesh_Test.shape)

(480, 10)
(27, 10)
(24, 10)


In [498]:
df_X=np.diff(df_X, axis=0)
df_Bangladesh_X=np.diff(df_Bangladesh_X, axis=0)
df_Bangladesh_Test=np.diff(df_Bangladesh_Test, axis=0)

In [499]:
print(df_X.shape)
print(df_Bangladesh_X.shape)
print(df_Bangladesh_Test.shape)

(479, 10)
(26, 10)
(23, 10)


In [500]:
df_Bangladesh_Y=np.diff(df_Bangladesh_Y,axis=0)
df_Y=np.diff(df_Y,axis=0)

In [501]:
print(df_Bangladesh_Y.shape)
print(df_Y.shape)

(26,)
(479,)


In [502]:
def predict_Linear(X,y,X_features_df,df_y,df_Bangladesh_Test,poly=None):
   
    model=LinearRegression()
    if (poly==True):
        poly = PolynomialFeatures(degree=3)
        X = poly.fit_transform(X)
        X_features_df = poly.fit_transform(X_features_df)
        df_Bangladesh_Test = poly.fit_transform(df_Bangladesh_Test)
        
    model.fit(X_features_df,df_y)
    predict_linear=model.predict(X)
    
    
    mse_linear = sklearn.metrics.mean_squared_error(y, predict_linear)
    rmse_linear = math.sqrt(mse_linear)
    from sklearn.metrics import r2_score,mean_absolute_error
    print("r2_score_linear",r2_score(y, predict_linear))
    print("mean_absolute_error_linear",mean_absolute_error(y, predict_linear))
    
    predict_future_linear=model.predict(df_Bangladesh_Test)  
   
    
    return mse_linear,rmse_linear,predict_linear,y_test,predict_future_linear

In [503]:
mse_linear,rmse_linear,predict_linear,y_test,predict_future_linear= predict_Linear(df_Bangladesh_X,df_Bangladesh_Y,df_X,df_Y,df_Bangladesh_Test,poly=True)


r2_score_linear -566522.0670441119
mean_absolute_error_linear 77081.90404534248


In [504]:
rmse_linear

106075.96426640575

In [505]:
mse_linear

11252110195.047789

In [506]:
predict_linear=predict_linear.astype(int)
predict_linear

array([-193421,   78876,   31592,   32641,  -56285,  -55232, -129991,
          9784, -168747,  -15254,  131022,  159638, -316280,  -33515,
        126116,   34370,  -26377,   18483,   56669,   -2307,  141235,
        -30339,   22849,   79083,   -9403,  -43237])

In [507]:
y_test

21    9
22    7
23    6
24    0
25    5
26    7
Name: Confirmed, dtype: int64

In [508]:
predict_future_linear=predict_future_linear.astype(int)
predict_future_linear

array([  56021, -123827,   23670,  110337,  -82703,   43776,  -60721,
        -59229,  220809,  -95065,  -84388,    3277,  -69865,   77814,
        103440,   33053,   12700,   15773,  -94622,  -32790,   76466,
        275013, -449573])